# 5: Motorized Travel Time

Model 5 relaxes the time constraints in Model 1 by specifying distinct time variables for the motorized and non-motorized modes based on our expectation that travelers are likely to be more sensitive to travel time by non-motorized modes. (pp. 111)r.

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import PX, P, X

m.utility_co[2] = P("ASC_SR2") + P("hhinc#2,3") * X("hhinc")
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#2,3") * X("hhinc")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

To only use the proper parameter for Motorized and Nonmotorized, we can take advantage of all motorized options being alternatives 1 through 4, and nonmotorized being 5 and 6.

In [ ]:
m.utility_ca = +(
    P("motorized_time") * X("(altid <= 4) * tottime")
    + P("nonmotorized_time") * X("(altid >4) * tottime")
    + PX("totcost")
)

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 5, Motorized Travel Times"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 20, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(20)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True)
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        "totcost",
        "motorized_time",
        "nonmotorized_time",
    ),
    (
        "Income",
        "hhinc.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_BIKE": -1.840892643878567,
    "ASC_SR2": -2.2622448341521775,
    "ASC_SR3P": -3.6771306979421228,
    "ASC_TRAN": -0.852957222474832,
    "ASC_WALK": 0.4685149210255025,
    "hhinc#2,3": -0.0015329803832402177,
    "hhinc#4": -0.005431498351689336,
    "hhinc#5": -0.012539862511200987,
    "hhinc#6": -0.009455692802202887,
    "motorized_time": -0.04307376568436681,
    "nonmotorized_time": -0.06856560413388568,
    "totcost": -0.0050037305794018135,
}
for k in expected_x:
    assert revealed_x[k] == approx(
        expected_x[k], 2e-2
    ), f"{k}, {revealed_x[k]/expected_x[k]}"